<a href="https://colab.research.google.com/github/ollyekhan/SocialMediaMiningFinalProject/blob/main/QA_Models_V1_By_Ali_Bangash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processing the data (TensorFlow)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
# !pip install datasets evaluate transformers[sentencepiece]

In [2]:
# !head dataset/postings.csv

In [3]:
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv("dataset/postings.csv")

# Dropping any NaN values in the about sections
df = df.dropna()
about_sections = df['job_skills'].tolist()

# Combine all about sections into one document
about_text = ' '.join(about_sections)

# Tokenize the text
vectorizer = CountVectorizer()
X = vectorizer.fit_transform([about_text])

# Get the list of terms and their frequencies
terms = vectorizer.get_feature_names_out()
term_counts = X.toarray().sum(axis=0)

# Create a dictionary of terms and their counts
term_count_dict = dict(zip(terms, term_counts))

# Sort the terms by their counts in descending order
sorted_terms = sorted(term_count_dict.items(), key=lambda x: x[1], reverse=True)

# Display the top N most popular skills
top_skills_count = 1000  # Change this number to display more or fewer skills
top_skills = sorted_terms[:top_skills_count]

# Convert the list of tuples to a DataFrame for easier visualization
top_skills_df = pd.DataFrame(top_skills, columns=['Skill', 'Frequency'])

top_skills_df.to_csv('dataset/top_skills.csv', index=False)

print("Top Skills saved to top_skills.csv")

# now tokenize
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_data = tokenizer(about_sections, padding=True, truncation=True, return_tensors="tf")

# creating input sequences
input_sequences = {
    'input_ids': tokenized_data['input_ids'],
    'attention_mask': tokenized_data['attention_mask']
}

# converting to tensorflow tensors
input_tensors = {key: tf.convert_to_tensor(value) for key, value in input_sequences.items()}

Top Skills saved to top_skills.csv


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]